In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from nltk import word_tokenize

# 2.2 特征提取二

## GloVe相似度

GloVe相似度是本节要提取的唯一一个特征。没有在2.1节中完成此项工作，是因为导入训练好的GloVe模型后，内存逼近Kernel的16GB上限，因此不得不开设一个新的Kernel专门提取GloVe相关的特征。

**GloVe是如何实现的？**

- 根据语料库构建共现矩阵$X$，矩阵中的元素$X_{ij}$代表单词$i$和上下文单词$j$在特定大小的上下文窗口内共同出现的次数。GloVe对上下文窗口内不同距离的单词赋予不同的权重，衰减函数为$decay = 1/d$，距离远的两个单词占总计数的权重更小。

- 构建词向量和共现矩阵的近似关系：

<center>$w_{i}^{T}\overset{\sim}w_{j}+b_{i}+\overset{\sim}b_{j}=log(X_{ij})$</center>

&ensp;&ensp;&ensp;其中，$w_{i}^{T}$和$\overset{\sim}w_{j}$是要最终求解的词向量，而$b_{i}$和$\overset{\sim}b_{j}$分别是两个词向量的偏置项。

- 构造损失函数：

<center>$J=\sum_{i,j=1}^{V}f(X_{ij})(w_{i}^{T}\overset{\sim}w_{j}+b_{i}+\overset{\sim}b_{j}-log(X_{ij}))^{2}$</center>

&ensp;&ensp;&ensp;上面的损失函数可以理解为由平方损失函数加上一个权重函数$f(X_{ij})$得到，$f(X_{ij})$可取为

$$f(x)=
\begin{cases}
(x/x_{max})^{\alpha}& x<x_{max}\\
1& otherwise
\end{cases}$$

&ensp;&ensp;&ensp;其中，$\alpha$的取值都是0.75，而$x_{max}$取值都是100。这样一来，越经常一起出现的单词权重越大，而当到达一定程度之后又不再增加，并且对于从来没有一起出现过的单词，也就是$X_{ij}=0$时，有$f(X_{if})$也为0。

- GloVe的学习同样基于梯度下降办法

**为什么不适用Word2Vec？**

使用Word2Vec有两种办法，一种是自己对训练集建立语料库，然后使用Gensim的Word2Vec函数实现，缺点是语料库太小，参数设定也缺乏指导，另一种办法是下载Google已经训练好的模型，但奈何相关页面最近两天好像崩溃了？于是本文转向了GloVe。

Kaggle的DataSet中刚好有本来需要在GloVe官网上下载的文本文件，导入Kernel后存放在embeddings_index中，它包含了绝大多数单词到他们的词向量的映射。

In [ ]:
f = open("../input/glove840b300dtxt/glove.840B.300d.txt", encoding="utf-8")
embeddings_index = {}
for line in f:
    values = line.split()
    word = "".join(values[:-300])   
    coefs = np.asarray(values[-300:], dtype="float32")
    embeddings_index[word] = coefs
f.close()
print("Found {} word vectors of glove.".format(len(embeddings_index)))

### 训练集

In [ ]:
train_orig = pd.read_csv("../input/quora-question-pairs-data-cleaning/train_orig.csv")
train_stop = pd.read_csv("../input/quora-question-pairs-data-cleaning/train_stop.csv")

train_orig.fillna("", inplace = True)
train_stop.fillna("", inplace = True)

train = pd.read_csv("../input/quora-question-pairs-feature-extraction-1/train.csv")
trainlabel = pd.read_csv("../input/quora-question-pairs-feature-extraction-1/trainlabel.csv")

定义函数get_word_vector得到每个问题包含的所有词向量。测试集或数据集里可能有一些非常罕见的单词，即便GloVe的训练数据量已经达到了60B级别也未能覆盖。对于这种特殊情况，判断两个问题是否都包含了这个词，如果是则有可能含有较多信息，用varity记录下来，否则忽略。

In [ ]:
def get_word_vector(row):
    wordlist1 = word_tokenize(row["question1"])
    wordlist2 = word_tokenize(row["question2"])
    
    rarity = 0  # 用于标记问题对是否含有非常罕见的词的特征
    
    embeddings_list1 = []
    for string in wordlist1:
        try:
            embeddings_list1.append(embeddings_index[string])
        except KeyError:
            if string in wordlist2:  # 如果两个问题包含这个词，令rarity=1，否则不进行处理
                rarity = 1
            else:
                pass          
    
    embeddings_list2 = []
    for string in wordlist2:
        try:
            embeddings_list2.append(embeddings_index[string])
        except KeyError:
            if string in wordlist1:
                rarity = 1
            else:
                pass  
    
    return pd.Series([embeddings_list1, embeddings_list2, rarity])

对去除停用词前和去除停用词后两个训练集都计算GloVe的相关特征，首先处理train_orig。

In [ ]:
vector_orig = train_orig.apply(get_word_vector, axis = 1)
vector_orig.columns = ["question1", "question2", "rarity"]

**通过词向量的平均值来衡量句子的语义，构建1-范数和2-范数以及夹角余弦值三个特征。**定义函数diff_word_vector完成计算。

In [ ]:
def diff_word_vector(row):
    mean1 = np.mean(np.array(row["question1"]), axis = 0)
    mean2 = np.mean(np.array(row["question2"]), axis = 0)
    diff = mean1 - mean2
    L1 = np.sum(np.abs(diff))  # np.linalg.norm不能对空的数组计算1-范数和2-范数，所以只能手动计算
    L2 = np.sum(diff ** 2) ** 0.5
    norm1 = np.sum(mean1 ** 2) ** 0.5
    norm2 = np.sum(mean2 ** 2) ** 0.5
    cos = np.sum(mean1 * mean2) / (norm1 * norm2)
    return pd.Series([L1, L2, cos])

In [ ]:
diff_vector_orig = vector_orig.apply(diff_word_vector, axis = 1)
features_vector_orig = pd.concat([diff_vector_orig, vector_orig["rarity"]], axis = 1)
features_vector_orig.columns = ["diff_word_vector_L1_orig", "diff_word_vector_L2_orig", 
                                "word_vector_cos_orig", "varity"]
train = pd.concat([train, features_vector_orig], axis = 1)

del vector_orig, diff_vector_orig, features_vector_orig

以同样的方法对去除停用词后的训练集计算相关特征。

In [ ]:
vector_stop = train_stop.apply(get_word_vector, axis = 1)
vector_stop.columns = ["question1", "question2", "rarity"]

diff_vector_stop = vector_stop.apply(diff_word_vector, axis = 1)
diff_vector_stop.columns = ["diff_word_vector_L1_stop", "diff_word_vector_L2_stop", 
                            "word_vector_cos_stop"]
train = pd.concat([train, diff_vector_stop], axis = 1)

del vector_stop, diff_vector_stop

In [ ]:
train.to_csv("train.csv", index = False)
trainlabel.to_csv("trainlabel.csv", index = False)

del train, trainlabel, train_orig, train_stop

### 测试集

In [ ]:
test_orig = pd.read_csv("../input/quora-question-pairs-data-cleaning/test_orig.csv")
test_orig.fillna("", inplace = True)

vector_orig = test_orig.apply(get_word_vector, axis = 1)
vector_orig.columns = ["question1", "question2", "rarity"]
del test_orig

diff_vector_orig = vector_orig.apply(diff_word_vector, axis = 1)
features_vector_orig = pd.concat([diff_vector_orig, vector_orig["rarity"]], axis = 1)
features_vector_orig.columns = ["diff_word_vector_L1_orig", "diff_word_vector_L2_orig", 
                                "word_vector_cos_orig", "varity"]
del vector_orig, diff_vector_orig

In [ ]:
test_stop = pd.read_csv("../input/quora-question-pairs-data-cleaning/test_stop.csv")
test_stop.fillna("", inplace = True)

vector_stop = test_stop.apply(get_word_vector, axis = 1)
vector_stop.columns = ["question1", "question2", "rarity"]
del test_stop, embeddings_index

diff_vector_stop = vector_stop.apply(diff_word_vector, axis = 1)
diff_vector_stop.columns = ["diff_word_vector_L1_stop", "diff_word_vector_L2_stop", 
                            "word_vector_cos_stop"]
del vector_stop

test = pd.read_csv("../input/quora-question-pairs-feature-extraction-1/test.csv")
test = pd.concat([test, features_vector_orig, diff_vector_stop], axis = 1)
del features_vector_orig, diff_vector_stop

In [ ]:
test.to_csv("test.csv", index = False)

第三部分模型训练见[Quora Question Pairs: XGBoost](https://www.kaggle.com/benjaminkz/quora-question-pairs-xgboost)。